In [1]:
from __future__ import absolute_import, division, print_function

import argparse
from datetime import datetime
from os import environ

import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.applications import MobileNet
from keras.callbacks import (EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard)
from keras.layers import Dense, GlobalAveragePooling2D
from keras.metrics import binary_accuracy, binary_crossentropy
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight

Using TensorFlow backend.


In [2]:
environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Shut up tensorflow!
print("tf : {}".format(tf.__version__))
print("keras : {}".format(keras.__version__))
print("numpy : {}".format(np.__version__))
print("pandas : {}".format(pd.__version__))

tf : 1.10.0
keras : 2.2.4
numpy : 1.16.0
pandas : 0.23.4


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=45,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    '/Users/sidgupta/Documents/mura/train',
    shuffle=True,  # otherwise we get distorted batch-wise metrics
    class_mode='binary',
    target_size=(224, 224),
    batch_size=32, )

val_datagen = ImageDataGenerator(rescale=1. / 255)
val_generator = val_datagen.flow_from_directory(
    '/Users/sidgupta/Documents/mura/valid',
    shuffle=True,  # otherwise we get distorted batch-wise metrics
    class_mode='binary',
    target_size=(224, 224),
    batch_size=32, )
    
classes = len(train_generator.class_indices)
assert classes > 0
assert classes is len(val_generator.class_indices)
n_of_train_samples = train_generator.samples
n_of_val_samples = val_generator.samples
    
base_model = MobileNet(input_shape=(224,224,3), weights='imagenet', include_top=False)
x = base_model.output  # Recast classification layer
# x = Flatten()(x)  # Uncomment for Resnet based models
x = GlobalAveragePooling2D(name='predictions_avg_pool')(x)  # comment for RESNET models
# n_classes; softmax for multi-class, sigmoid for binary
x = Dense(1, activation='sigmoid', name='predictions')(x)
model = Model(inputs=base_model.input, outputs=x)    

now_iso = datetime.now().strftime('%Y-%m-%dT%H:%M:%S%z')
    
checkpoint = ModelCheckpoint(filepath='./models/MobileNet.hdf5', verbose=1, save_best_only=True)
early_stop = EarlyStopping(patience=20)
tensorboard = TensorBoard(log_dir='./logs/MobileNet/{}/'.format(now_iso))
# reduce_lr = ReduceLROnPlateau(factor=0.03, cooldown=0, patience=args.lr_wait, min_lr=0.1e-6)
callbacks = [checkpoint, tensorboard, checkpoint]
    
weights = class_weight.compute_class_weight('balanced', np.unique(train_generator.classes), train_generator.classes)
weights = {0: weights[0], 1: weights[1]}
# for layer in base_model.layers:
#     layer.set_trainable = False

# print(model.summary())
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name)
if '':
    model.load_weights('')
    for layer in model.layers:
        layer.set_trainable = True    
    
    
model.compile(
    optimizer=Adam(lr=10, decay=1e-4),
    # optimizer=SGD(lr=args.lr, decay=args.decay,momentum=args.momentum, nesterov=True),
    loss=binary_crossentropy,
    metrics=[binary_accuracy], )

model_out = model.fit_generator(
    train_generator,
    steps_per_epoch=n_of_train_samples//32,
    epochs=120,
    validation_data=val_generator,
    validation_steps=n_of_val_samples//32,
    class_weight=weights,
    workers=4,
    use_multiprocessing=True,
    callbacks=callbacks)

Found 36808 images belonging to 2 classes.
Found 3197 images belonging to 2 classes.
Epoch 1/120
1150/1150 [==============================] - 221158s 192s/step - loss: nan - binary_accuracy: 3.8043e-04 - val_loss: nan - val_binary_accuracy: 0.0000e+00